In [27]:
import setup

setup.init_django()

In [28]:
from market.models import StockQuote
from django.db.models import Avg, F, RowRange, Window
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [ ]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=days_ago)
end_date = now

qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

In [30]:
total = 0
for obj in qs:
    total += obj.close_price

In [ ]:
total / qs.count()

In [ ]:
qs.values("company").annotate(avg_price=Avg("close_price"))

In [ ]:
count = 5
ticker = "META"
rolling_qs = list(qs.filter(company__ticker=ticker).order_by("-time")[:count])
rolling_qs.reverse()
rolling_qs

In [ ]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = total / (i + 1)
    print(i + 1, obj.id, obj.close_price, avg)

In [35]:
qs = StockQuote.objects.filter(
    company__ticker=ticker, time__range=(start_date, end_date)
)

In [36]:
frame_start = -(count - 1)
ma_val = qs.annotate(
    ma=Window(
        expression=Avg("close_price"),
        order_by=F("time").asc(),
        partition_by=[],
        frame=RowRange(start=frame_start, end=0),
    )
).order_by("-time")

In [ ]:
for obj in ma_val[:5]:
    print(obj.id, obj.close_price, obj.ma, obj.time)

In [ ]:
frame_start = -(count - 1)
ma_vals = (
    qs.annotate(
        ma_5=Window(
            expression=Avg("close_price"),
            order_by=F("time").asc(),
            partition_by=[],
            frame=RowRange(start=-4, end=0),
        ),
        ma_20=Window(
            expression=Avg("close_price"),
            order_by=F("time").asc(),
            partition_by=[],
            frame=RowRange(start=-19, end=0),
        ),
    )
    .order_by("-time")
    .first()
)

ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20